In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:0.9.1` // Not required since almond 0.7.0 (will be automatically added when importing spark)
import $ivy.`org.apache.spark::spark-mllib:2.4.4`

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                               // Not required since almond 0.7.0 (will be automatically added when importing spark)

import $ivy.$                                    

In [2]:
import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType, ArrayType, DataType}

import org.apache.spark.SparkContext

import org.apache.spark.SparkContext._

import org.apache.spark.SparkConf

import org.apache.spark.sql.SparkSession

import org.apache.spark.sql.functions._

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType, ArrayType, DataType}

In [3]:
import org.apache.log4j.{Level, Logger}
val rootLogger = Logger.getRootLogger()
rootLogger.setLevel(Level.WARN)

import org.apache.log4j.{Level, Logger}

rootLogger: Logger = org.apache.log4j.spi.RootLogger@5ba81de9

In [4]:
val conf = new SparkConf().setMaster("local[*]").setAppName("read from localhost")
val sc = new SparkContext(conf)
val spark = SparkSession.builder.config(sc.getConf).getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
20/05/14 15:33:33 WARN Utils: Your hostname, Dis-iMac.local resolves to a loopback address: 127.0.0.1; using 10.1.1.163 instead (on interface en1)
20/05/14 15:33:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
20/05/14 15:33:34 INFO SparkContext: Running Spark version 2.4.0
20/05/14 15:33:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
20/05/14 15:33:34 INFO SparkContext: Submitted application: read from localhost
20/05/14 15:33:34 INFO SecurityManager: Changing view acls to: diyang
20/05/14 15:33:34 INFO SecurityManager: Changing modify acls to: diyang
20/05/14 15:33:34 INFO SecurityManager: Changing view acls groups to: 
20/05/14 15:33:34 INFO SecurityManager: Changing modify acls groups to: 
20/05/14 15:33:34 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with

conf: SparkConf = org.apache.spark.SparkConf@4f53bf35
sc: SparkContext = org.apache.spark.SparkContext@2158f4e3
spark: SparkSession = org.apache.spark.sql.SparkSession@5f2d20f5

In [5]:
val file_location = "hdfs://localhost:8020/user/webhose/discussions/"
val file_type = "json"

file_location: String = "hdfs://localhost:8020/user/webhose/discussions/"
file_type: String = "json"

In [6]:
def flattenDataframe(df: DataFrame): DataFrame = {
    //getting all the fields from schema
    val fields = df.schema.fields
    val fieldNames = fields.map(x => x.name)
    //length shows the number of fields inside dataframe
    val length = fields.length
    for (i <- 0 to fields.length - 1) {
      val field = fields(i)
      val fieldtype = field.dataType
      val fieldName = field.name
      fieldtype match {
        case arrayType: ArrayType =>
          val fieldName1 = fieldName
          val fieldNamesExcludingArray = fieldNames.filter(_ != fieldName1)
          val fieldNamesAndExplode = fieldNamesExcludingArray ++ Array(s"explode_outer($fieldName1) as $fieldName1")
          //val fieldNamesToSelect = (fieldNamesExcludingArray ++ Array(s"$fieldName1.*"))
          val explodedDf = df.selectExpr(fieldNamesAndExplode: _*)
          return flattenDataframe(explodedDf)

        case structType: StructType =>
          val childFieldnames = structType.fieldNames.map(childname => fieldName + "." + childname)
          val newfieldNames = fieldNames.filter(_ != fieldName) ++ childFieldnames
          val renamedcols = newfieldNames.map(x => (col(x.toString()).as(x.toString().replace(".", "_").replace("$", "_").replace("__", "_").replace(" ", "").replace("-", ""))))
          val explodedf = df.select(renamedcols: _*)
          return flattenDataframe(explodedf)
        case _ =>
      }
    }
    df
  }

defined function flattenDataframe

In [7]:
val jsonFullDFSchemaString = spark.read.format(file_type).load(file_location).schema.json
val jsonFullDFSchemaStructType = DataType.fromJson(jsonFullDFSchemaString).asInstanceOf[StructType]
val df = spark.read.schema(jsonFullDFSchemaStructType).json(file_location);

20/05/14 15:33:41 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir ('file:/Users/diyang/Quatitative-Research/DPC_DDI_PROJECT/MITdatatholn/spark-warehouse').
20/05/14 15:33:41 INFO SharedState: Warehouse path is 'file:/Users/diyang/Quatitative-Research/DPC_DDI_PROJECT/MITdatatholn/spark-warehouse'.
20/05/14 15:33:42 INFO StateStoreCoordinatorRef: Registered StateStoreCoordinator endpoint
20/05/14 15:33:44 INFO FileSourceStrategy: Pruning directories with: 
20/05/14 15:33:44 INFO FileSourceStrategy: Post-Scan Filters: 
20/05/14 15:33:44 INFO FileSourceStrategy: Output Data Schema: struct<value: string>
20/05/14 15:33:44 INFO FileSourceScanExec: Pushed Filters: 
20/05/14 15:33:45 INFO CodeGenerator: Code generated in 245.793729 ms
20/05/14 15:33:45 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 220.2 KB, free 912.1 MB)
20/05/14 15:33:45 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memor

jsonFullDFSchemaString: String = "{\"type\":\"struct\",\"fields\":[{\"name\":\"author\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"crawled\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"entities\",\"type\":{\"type\":\"struct\",\"fields\":[{\"name\":\"locations\",\"type\":{\"type\":\"array\",\"elementType\":\"string\",\"containsNull\":true},\"nullable\":true,\"metadata\":{}},{\"name\":\"organizations\",\"type\":{\"type\":\"array\",\"elementType\":\"string\",\"containsNull\":true},\"nullable\":true,\"metadata\":{}},{\"name\":\"persons\",\"type\":{\"type\":\"array\",\"elementType\":\"string\",\"containsNull\":true},\"nullable\":true,\"metadata\":{}}]},\"nullable\":true,\"metadata\":{}},{\"name\":\"external_links\",\"type\":{\"type\":\"array\",\"elementType\":\"string\",\"containsNull\":true},\"nullable\":true,\"metadata\":{}},{\"name\":\"highlightText\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"highlightTitle\",\"type\

In [8]:
val tableSchemaDF = flattenDataframe(df)

tableSchemaDF: DataFrame = [author: string, crawled: string ... 41 more fields]

In [ ]:
val table_5G = tableSchemaDF.select(col("thread_title"))
.filter((col("thread_title").contains("coronavirus")) && (col("thread_title").contains("5G")))
.distinct

In [ ]:
table_5G.count()

In [ ]:
table_5G.show()

In [ ]:
val table_5G_2 = tableSchemaDF
.withColumn("thread_title", lower(col("thread_title")))
.filter((col("thread_title").contains("virus")) && (col("thread_title").contains("5g")))
.select("thread_title")
.distinct

In [ ]:
table_5G_2.show(100, false)

In [ ]:
table_5G_2.count()

In [11]:
val table_text_5g = tableSchemaDF
.withColumn("text", lower(col("text")))
.filter((col("text").contains("virus")) && (col("text").contains("5g")))
.distinct

table_text_5g: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [author: string, crawled: string ... 41 more fields]

In [ ]:
table_text_5g.show()

In [12]:
table_text_5g.count()

20/05/14 15:35:02 INFO FileSourceStrategy: Pruning directories with: 
20/05/14 15:35:02 INFO FileSourceStrategy: Post-Scan Filters: Contains(lower(text#52), virus),Contains(lower(text#52), 5g)
20/05/14 15:35:02 INFO FileSourceStrategy: Output Data Schema: struct<author: string, crawled: string, entities: struct<locations: array<string>, organizations: array<string>, persons: array<string> ... 1 more fields>, external_links: array<string>, highlightText: string ... 15 more fields>
20/05/14 15:35:02 INFO FileSourceScanExec: Pushed Filters: 
20/05/14 15:35:02 INFO CodeGenerator: Code generated in 15.220601 ms
20/05/14 15:35:02 INFO HashAggregateExec: spark.sql.codegen.aggregate.map.twolevel.enabled is set to true, but current version of codegened fast hashmap does not support this aggregate.
20/05/14 15:35:02 INFO CodeGenerator: Code generated in 64.863603 ms
20/05/14 15:35:02 INFO HashAggregateExec: spark.sql.codegen.aggregate.map.twolevel.enabled is set to true, but current version of c

20/05/14 15:35:06 INFO ContextCleaner: Cleaned accumulator 77
20/05/14 15:35:06 INFO ContextCleaner: Cleaned accumulator 83
20/05/14 15:35:06 INFO ContextCleaner: Cleaned accumulator 65
20/05/14 15:35:06 INFO ContextCleaner: Cleaned accumulator 39
20/05/14 15:35:06 INFO ContextCleaner: Cleaned accumulator 35
20/05/14 15:35:06 INFO ContextCleaner: Cleaned accumulator 45
20/05/14 15:35:06 INFO ContextCleaner: Cleaned accumulator 54
20/05/14 15:35:06 INFO ContextCleaner: Cleaned accumulator 66
20/05/14 15:35:06 INFO ContextCleaner: Cleaned accumulator 80
20/05/14 15:35:06 INFO ContextCleaner: Cleaned accumulator 73
20/05/14 15:35:06 INFO ContextCleaner: Cleaned accumulator 58
20/05/14 15:35:06 INFO ContextCleaner: Cleaned accumulator 74
20/05/14 15:35:06 INFO ContextCleaner: Cleaned accumulator 56
20/05/14 15:35:06 INFO ContextCleaner: Cleaned accumulator 49
20/05/14 15:35:06 INFO ContextCleaner: Cleaned accumulator 43
20/05/14 15:35:06 INFO ContextCleaner: Cleaned accumulator 34
20/05/14

20/05/14 15:35:09 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 1 ms
20/05/14 15:35:09 INFO ShuffleBlockFetcherIterator: Getting 4 non-empty blocks including 4 local blocks and 0 remote blocks
20/05/14 15:35:09 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
20/05/14 15:35:09 INFO Executor: Finished task 4.0 in stage 4.0 (TID 20). 4058 bytes result sent to driver
20/05/14 15:35:09 INFO TaskSetManager: Starting task 6.0 in stage 4.0 (TID 22, localhost, executor driver, partition 6, ANY, 7235 bytes)
20/05/14 15:35:09 INFO TaskSetManager: Finished task 4.0 in stage 4.0 (TID 20) in 22 ms on localhost (executor driver) (3/200)
20/05/14 15:35:09 INFO Executor: Running task 6.0 in stage 4.0 (TID 22)
20/05/14 15:35:09 INFO ShuffleBlockFetcherIterator: Getting 4 non-empty blocks including 4 local blocks and 0 remote blocks
20/05/14 15:35:09 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 1 ms
20/05/14 15:35:09 INFO Executor: Finished task 5.0 in 

20/05/14 15:35:09 INFO ShuffleBlockFetcherIterator: Getting 4 non-empty blocks including 4 local blocks and 0 remote blocks
20/05/14 15:35:09 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
20/05/14 15:35:09 INFO ShuffleBlockFetcherIterator: Getting 4 non-empty blocks including 4 local blocks and 0 remote blocks
20/05/14 15:35:09 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
20/05/14 15:35:09 INFO Executor: Finished task 10.0 in stage 4.0 (TID 26). 4101 bytes result sent to driver
20/05/14 15:35:09 INFO Executor: Finished task 17.0 in stage 4.0 (TID 33). 4058 bytes result sent to driver
20/05/14 15:35:10 INFO TaskSetManager: Starting task 18.0 in stage 4.0 (TID 34, localhost, executor driver, partition 18, ANY, 7235 bytes)
20/05/14 15:35:10 INFO TaskSetManager: Finished task 10.0 in stage 4.0 (TID 26) in 130 ms on localhost (executor driver) (15/200)
20/05/14 15:35:10 INFO Executor: Running task 18.0 in stage 4.0 (TID 34)
20/05/14 15:35:10 INFO T

20/05/14 15:35:10 INFO TaskSetManager: Finished task 29.0 in stage 4.0 (TID 45) in 13 ms on localhost (executor driver) (27/200)
20/05/14 15:35:10 INFO Executor: Running task 30.0 in stage 4.0 (TID 46)
20/05/14 15:35:10 INFO Executor: Running task 28.0 in stage 4.0 (TID 44)
20/05/14 15:35:10 INFO ShuffleBlockFetcherIterator: Getting 4 non-empty blocks including 4 local blocks and 0 remote blocks
20/05/14 15:35:10 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 1 ms
20/05/14 15:35:10 INFO Executor: Finished task 27.0 in stage 4.0 (TID 43). 4101 bytes result sent to driver
20/05/14 15:35:10 INFO TaskSetManager: Starting task 31.0 in stage 4.0 (TID 47, localhost, executor driver, partition 31, ANY, 7235 bytes)
20/05/14 15:35:10 INFO TaskSetManager: Finished task 27.0 in stage 4.0 (TID 43) in 100 ms on localhost (executor driver) (28/200)
20/05/14 15:35:10 INFO Executor: Running task 31.0 in stage 4.0 (TID 47)
20/05/14 15:35:10 INFO Executor: Finished task 25.0 in stage 4.0 (

...

20/05/14 15:35:11 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
20/05/14 15:35:11 INFO Executor: Running task 160.0 in stage 4.0 (TID 176)
20/05/14 15:35:11 INFO ShuffleBlockFetcherIterator: Getting 4 non-empty blocks including 4 local blocks and 0 remote blocks
20/05/14 15:35:11 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
20/05/14 15:35:11 INFO Executor: Finished task 160.0 in stage 4.0 (TID 176). 4058 bytes result sent to driver
20/05/14 15:35:11 INFO TaskSetManager: Starting task 167.0 in stage 4.0 (TID 183, localhost, executor driver, partition 167, ANY, 7235 bytes)
20/05/14 15:35:11 INFO TaskSetManager: Finished task 160.0 in stage 4.0 (TID 176) in 46 ms on localhost (executor driver) (164/200)
20/05/14 15:35:11 INFO Executor: Running task 167.0 in stage 4.0 (TID 183)
20/05/14 15:35:11 INFO ShuffleBlockFetcherIterator: Getting 4 non-empty blocks including 4 local blocks and 0 remote blocks
20/05/14 15:35:11 INFO ShuffleBlockFetcherIterat

20/05/14 15:35:11 INFO TaskSetManager: Starting task 179.0 in stage 4.0 (TID 195, localhost, executor driver, partition 179, ANY, 7235 bytes)
20/05/14 15:35:11 INFO TaskSetManager: Finished task 172.0 in stage 4.0 (TID 188) in 51 ms on localhost (executor driver) (176/200)
20/05/14 15:35:11 INFO Executor: Finished task 178.0 in stage 4.0 (TID 194). 4058 bytes result sent to driver
20/05/14 15:35:11 INFO TaskSetManager: Starting task 180.0 in stage 4.0 (TID 196, localhost, executor driver, partition 180, ANY, 7235 bytes)
20/05/14 15:35:11 INFO TaskSetManager: Finished task 178.0 in stage 4.0 (TID 194) in 12 ms on localhost (executor driver) (177/200)
20/05/14 15:35:11 INFO Executor: Running task 180.0 in stage 4.0 (TID 196)
20/05/14 15:35:11 INFO Executor: Running task 179.0 in stage 4.0 (TID 195)
20/05/14 15:35:11 INFO ShuffleBlockFetcherIterator: Getting 4 non-empty blocks including 4 local blocks and 0 remote blocks
20/05/14 15:35:11 INFO ShuffleBlockFetcherIterator: Started 0 remote

20/05/14 15:35:11 INFO Executor: Running task 191.0 in stage 4.0 (TID 207)
20/05/14 15:35:11 INFO Executor: Finished task 190.0 in stage 4.0 (TID 206). 4058 bytes result sent to driver
20/05/14 15:35:11 INFO Executor: Finished task 187.0 in stage 4.0 (TID 203). 4058 bytes result sent to driver
20/05/14 15:35:11 INFO ShuffleBlockFetcherIterator: Getting 4 non-empty blocks including 4 local blocks and 0 remote blocks
20/05/14 15:35:11 INFO TaskSetManager: Starting task 192.0 in stage 4.0 (TID 208, localhost, executor driver, partition 192, ANY, 7235 bytes)
20/05/14 15:35:11 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 2 ms
20/05/14 15:35:11 INFO TaskSetManager: Starting task 193.0 in stage 4.0 (TID 209, localhost, executor driver, partition 193, ANY, 7235 bytes)
20/05/14 15:35:11 INFO TaskSetManager: Finished task 190.0 in stage 4.0 (TID 206) in 15 ms on localhost (executor driver) (189/200)
20/05/14 15:35:11 INFO TaskSetManager: Finished task 187.0 in stage 4.0 (TID 203

20/05/14 15:35:12 INFO Executor: Running task 0.0 in stage 5.0 (TID 216)
20/05/14 15:35:12 INFO ShuffleBlockFetcherIterator: Getting 200 non-empty blocks including 200 local blocks and 0 remote blocks
20/05/14 15:35:12 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 1 ms
20/05/14 15:35:12 INFO Executor: Finished task 0.0 in stage 5.0 (TID 216). 1749 bytes result sent to driver
20/05/14 15:35:12 INFO TaskSetManager: Finished task 0.0 in stage 5.0 (TID 216) in 30 ms on localhost (executor driver) (1/1)
20/05/14 15:35:12 INFO TaskSchedulerImpl: Removed TaskSet 5.0, whose tasks have all completed, from pool 
20/05/14 15:35:12 INFO DAGScheduler: ResultStage 5 (count at cmd11.sc:1) finished in 0.035 s
20/05/14 15:35:12 INFO DAGScheduler: Job 2 finished: count at cmd11.sc:1, took 9.302109 s


res11: Long = 6010L